In [1]:
import flywheel
import pandas as pd
from datetime import datetime

fw = flywheel.Client()
gear = fw.lookup('gears/wscore-ct-heatmap/0.4.0_0.4.0') 
project_id = '60fef55e60ec55d1b0e0741e'
collection_id = '618d6fb6bb845b310730d8f8'
project = fw.get(project_id) 
collection = fw.get(collection_id) 
gear = 'antsct-aging-fw'
gear_fw = fw.lookup('gears/antsct-aging-fw/0.3.1_0.3.3')

## Identify sessions without ANTs-CT runs

In [46]:
collection_sessions = [fw.get_session(x.id) for x in fw.get_collection_sessions(collection_id)]

In [47]:
# Create a data dict:
data_dict = {'subject':[],'session':[], 'project':[],'run':[],'status':[], 'label': []}

# Iterate over sessions
for session in collection_sessions:

    # Because we want information off the sessions's analyses, we need to reload
    # The container to make sure we have all the metadata.
    session = session.reload()
    sub_label = session.subject.label
    ses_label = session.label
    project = fw.get(session.parents['project']).label
    # Any analyses on this session will be stored as a list:
    analyses = session.analyses

    # If there are no analyses containers, we know that this gear was not run
    if len(analyses) == 0:
        run = 'False'
        status = 'NA'
        label = 'NA'
    else:
        # Loop through the analyses
        matches = [asys for asys in analyses if asys.gear_info.get('name') == gear]
        # If there are no matches, the gear didn't run
        if len(matches) == 0:
            run = 'False'
            status = 'NA'
            label = 'NA'
        # If there is one match, that's our target
        elif len(matches) == 1:
            run = 'True'
            status = matches[0].job.get('state')
            label = matches[0].label
        # If there are more than one matches (due to reruns), take the most recent run.
        # This behavior may be modified to whatever suits your needs
        else:
            last_run_date = max([asys.created for asys in matches])
            last_run_analysis = [asys for asys in matches if asys.created == last_run_date]

            # There should only be one exact match
            last_run_analysis = last_run_analysis[0]

            run = 'True'
            status = last_run_analysis.job.get('state')
            label = last_run_analysis.label

    # Populate our data dict - remember that each key in the data dict must be updated
    # So that the length of our lists stays the same
    data_dict['subject'].append(sub_label)
    data_dict['session'].append(ses_label)
    data_dict['run'].append(run)
    data_dict['status'].append(status)
    data_dict['label'].append(label)
    data_dict['project'].append(project)

# Now create a data frame
df = pd.DataFrame.from_dict(data_dict)
df

,subject,session,project,run,status,label
0,123894,123894x20190327x3T,NACC-SC,True,complete,antsct_2021-05-13_WT_PVS
1,100049,100049x20160621x3T,NACC-SC,True,complete,antsct_2021-05-13_WT_PVS
2,100113,100113x20160620x3T,NACC-SC,True,complete,100113_100113x20160620x3T_antsct-aging-fw_0.3....
3,120937,120937x20160711x3T,NACC-SC,True,complete,120937_120937x20160711x3T_antsct-aging-fw_0.3....
4,119851,119851x20170606x3T,NACC-SC,True,complete,antsct_2021-05-13_WT_PVS
5,121105,121105x20170427x3T,NACC-SC,True,complete,antsct-aging-fw_2021-07-09_WT
6,122005,122005x20171213x3T,NACC-SC,True,complete,antsct-aging-fw_2021-07-09_WT
7,100957,100957_20110426,PMC-CLINICAL,True,complete,100957_100957_20110426_antsct-aging-fw_0.3.1_0...
8,100551,20081008x1021,HUP6,True,running,100551_20081008x1021_antsct-aging-fw_0.3.1_0.3...
9,100978,20041103x1130,HUP6,True,complete,100978_20041103x1130_antsct-aging-fw_0.2.2_0.3...


In [43]:
# Append a timestamp to our csv name so it won't overwrite anything when we upload it to flywheel
time_fmt = '%m-%d-%Y_%H-%M-%S'
time_string = datetime.now().strftime(time_fmt)
csv_out = f'../run_reports/{gear}_RunReport_{time_string}.csv'

df.to_csv(csv_out,index=False)

collection.upload_file(csv_out)

## Run ANTs-CT analyses for Neuroprint Validation

In [6]:
def get_bids_nifti(acq):
    '''
        Returns a BIDS-format NIfTI T1 image from an acquisition.
        11/16/2021 - Removed the BIDs requirement as some haven't been converted yet - WT
    '''
    bids_niftis = [f for f in acq.files if ('info' in f.keys() and "nii" in f.name)]
    if len(bids_niftis) == 1:
        return(bids_niftis.pop())
    elif len(bids_niftis) > 1:
        return(bids_niftis)
    else:
        return(None)

def get_nacc_t1_file(sess):
    '''
        Function to pick the most desirable T1 file out of several in a session. Very, very FTDC-specific.
    '''
    
    t1_acq = []
    acqlist = sess.acquisitions()
    for acq in acqlist:
        if any(['T1' in f.classification['Measurement'] for f in acq.files \
            if 'Measurement' in f.classification.keys()]):
                t1_acq.append(acq)
    
    t1_file = None
    
    for acq in t1_acq:
        lab = acq.label.lower()
        if ("t1" in lab) and not ("nd" in lab):
            t1_file = get_bids_nifti(acq)
            return(t1_file)
      
    t1_file = get_bids_nifti(t1_acq.pop())
    return(t1_file)

def get_ftdc_t1_file(sess):
    '''
        Function to pick the most desirable T1 file out of several in a session. Very, very FTDC-specific.
    '''
    #is_t1 = [any(['T1' in f.classification['Measurement'] for f in a.files \
    #    if 'Measurement' in f.classification.keys()]) for a in sess.acquisitions()]
    #t1_acq = [a for (a, v) in zip(sess.acquisitions(), is_t1) if v]
    
    t1_acq = []
    acqlist = sess.acquisitions()
    for acq in acqlist:
        if any(['T1' in f.classification['Measurement'] for f in acq.files \
            if 'Measurement' in f.classification.keys()]):
                t1_acq.append(acq)
    
    t1_file = None
    
    for acq in t1_acq:
        lab = acq.label.lower()
        if ("vnav" in lab) and ("moco" in lab) and ("rms" in lab) and not ("nd" in lab):
            t1_file = get_bids_nifti(acq)
            return(t1_file)
    
    for acq in t1_acq:
        lab = acq.label.lower()
        if ("vnav" in lab) and ("rms" in lab) and not ("nd" in lab):
            t1_file = get_bids_nifti(acq)
            return(t1_file)
    
    for acq in t1_acq:
        lab = acq.label.lower()
        if ("ax" in lab) and ("mprage" in lab):
            t1_file = get_bids_nifti(acq)
            return(t1_file)
    
    for acq in t1_acq:
        lab = acq.label.lower()
        if ("sag" in lab) and ("mprage" in lab):
            t1_file = get_bids_nifti(acq)
            return(t1_file)
    
    t1_file = get_bids_nifti(t1_acq.pop())
    return(t1_file)

In [45]:
# date for label
x = datetime.now()
datestr = '%s-%s-%s_%s:%s' % (x.year, x.month, x.day, x.hour, x.minute)

for ses in collection_sessions:
    # has ants been run on this session?
    run = df.loc[df.session==ses.label]['run'].item()
    # if not, run it
    if run == 'False':
        project_label = fw.get(ses.project).label
        if project_label == 'NACC-SC':
            t1_file = get_nacc_t1_file(ses)
        elif (project_label=='HUP6') or (project_label=='PMC-CLINICAL'):
            t1_file = get_ftdc_t1_file(ses)
        print(t1_file.name)

        acq = t1_file.parent.label
        asys_label = f'{ses.subject.label}_{ses.label}_antsct-aging-fw_0.3.1_0.3.3_{datestr}'
        print(f"RUNNING {asys_label} ON {ses.label}'S {acq} ACQUISITION...")
        # Run the gear
        inputs = {'t1_anatomy': t1_file}
        config = {'denoise': True, 'num-threads': 0, 'trim-neck-mode': 'mask', 'run-quick': False}
        analysis_id = gear_fw.run(analysis_label=asys_label, config=config, inputs=inputs, destination=ses)

t1_mpr_AX_MPRAGE_2.nii.gz
RUNNING 100551_20081008x1021_antsct-aging-fw_0.3.1_0.3.3_2021-11-24_12:56 ON 20081008x1021'S t1_mpr_AX_MPRAGE ACQUISITION...
t1_mpr_AX_MPRAGE_2.nii.gz
RUNNING 101841x02_20090106x1214_antsct-aging-fw_0.3.1_0.3.3_2021-11-24_12:56 ON 20090106x1214'S t1_mpr_AX_MPRAGE ACQUISITION...
t1_mpr_AX_MPRAGE_2.nii.gz
RUNNING 104190_20091109x1206_antsct-aging-fw_0.3.1_0.3.3_2021-11-24_12:56 ON 20091109x1206'S t1_mpr_AX_MPRAGE ACQUISITION...
t1_mpr_AX_MPRAGE_2.nii.gz
RUNNING 105223_20091103x1415_antsct-aging-fw_0.3.1_0.3.3_2021-11-24_12:56 ON 20091103x1415'S t1_mpr_AX_MPRAGE ACQUISITION...
t1_mpr_AX_MPRAGE_2.nii.gz
RUNNING 108790_20090218x1235_antsct-aging-fw_0.3.1_0.3.3_2021-11-24_12:56 ON 20090218x1235'S t1_mpr_AX_MPRAGE ACQUISITION...
t1_mpr_AX_MPRAGE_2.nii.gz
RUNNING 109198_20091201x1029_antsct-aging-fw_0.3.1_0.3.3_2021-11-24_12:56 ON 20091201x1029'S t1_mpr_AX_MPRAGE ACQUISITION...
t1_mpr_AX_MPRAGE_7.nii.gz
RUNNING 115001_20090331x0933_antsct-aging-fw_0.3.1_0.3.3_2021-11-

Ran manually on 122417, because although 'run' was True, the original run failed and was not re-run by the above code block.

### Attempt to re-run still pending jobs (Nov 18 11:43)


In [17]:
# date for label
x = datetime.now()
datestr = '%s-%s-%s_%s:%s' % (x.year, x.month, x.day, x.hour, x.minute)

for ses in collection_sessions:
    # what's the state of this job
    status = df.loc[df.session==ses.label]['status'].item()
    # if still pending, delete and try again
    if status in ('pending', 'cancelled'):
        # delete the analysis
        label = df.loc[df.session==ses.label]['label'].item()
        pending_asys = [a for a in ses.analyses if a.label==label]
        if pending_asys and len(pending_asys) > 0:
            a = pending_asys[0]
            fw.delete_session_analysis(session.id, a.id)
            print(f"DELETING {label} FROM {ses.label}...")
        
        # get the T1 input
        project_label = fw.get(ses.project).label
        if project_label == 'NACC-SC':
            t1_file = get_nacc_t1_file(ses)
        elif (project_label=='HUP6') or (project_label=='PMC-CLINICAL'):
            t1_file = get_ftdc_t1_file(ses)
        # prep 
        acq = t1_file.parent.label
        asys_label = f'{ses.subject.label}_{ses.label}_antsct-aging-fw_0.3.1_0.3.3_{datestr}'
        print(f"RUNNING {asys_label} ON {ses.label}'S {acq} ACQUISITION...")
        # Run the gear
        inputs = {'t1_anatomy': t1_file}
        config = {'denoise': True, 'num-threads': 0, 'trim-neck-mode': 'mask', 'run-quick': False}
        analysis_id = gear_fw.run(analysis_label=asys_label, config=config, inputs=inputs, destination=ses)

RUNNING 100551_20081008x1021_antsct-aging-fw_0.3.1_0.3.3_2021-11-18_12:3 ON 20081008x1021'S t1_mpr_AX_MPRAGE ACQUISITION...
RUNNING 101841x02_20090106x1214_antsct-aging-fw_0.3.1_0.3.3_2021-11-18_12:3 ON 20090106x1214'S t1_mpr_AX_MPRAGE ACQUISITION...
RUNNING 104190_20091109x1206_antsct-aging-fw_0.3.1_0.3.3_2021-11-18_12:3 ON 20091109x1206'S t1_mpr_AX_MPRAGE ACQUISITION...
RUNNING 105223_20091103x1415_antsct-aging-fw_0.3.1_0.3.3_2021-11-18_12:3 ON 20091103x1415'S t1_mpr_AX_MPRAGE ACQUISITION...
RUNNING 108790_20090218x1235_antsct-aging-fw_0.3.1_0.3.3_2021-11-18_12:3 ON 20090218x1235'S t1_mpr_AX_MPRAGE ACQUISITION...
RUNNING 109198_20091201x1029_antsct-aging-fw_0.3.1_0.3.3_2021-11-18_12:3 ON 20091201x1029'S t1_mpr_AX_MPRAGE ACQUISITION...
RUNNING 115001_20090331x0933_antsct-aging-fw_0.3.1_0.3.3_2021-11-18_12:3 ON 20090331x0933'S t1_mpr_AX_MPRAGE ACQUISITION...
RUNNING 115264_20090304x1208_antsct-aging-fw_0.3.1_0.3.3_2021-11-18_12:3 ON 20090304x1208'S t1_mpr_AX_MPRAGE ACQUISITION...
RUNNI